In [1]:
import numpy as np
import pandas as pd
import pm4py
from scipy import stats

In [6]:
import import_ipynb
from logEnrichment import enrichment
from filteringOperations import filterCasesByActivities
from interestingness_functions import compute_interestingness_kl_divergence,compute_interestingness_variance

In [4]:
log = pm4py.read_xes('DomesticDeclarations.xes_')
dataframe = pm4py.convert_to_dataframe(log)

In [5]:
log_ct=enrichment(dataframe,"case:id","time:timestamp","concept:name")

In [5]:
log_ct

,id,org:resource,concept:name,time:timestamp,org:role,case:id,case:concept:name,case:BudgetNumber,case:DeclarationNumber,case:Amount,Initial Activity,Final Activity,Cycle time,Transition
0,st_step 86794_0,STAFF MEMBER,Declaration SUBMITTED by EMPLOYEE,2017-01-09 09:49:50+01:00,EMPLOYEE,declaration 86791,declaration 86791,budget 86566,declaration number 86792,26.851205,Declaration SUBMITTED by EMPLOYEE,Declaration FINAL_APPROVED by SUPERVISOR,97.966667,"(Declaration SUBMITTED by EMPLOYEE, Declaration FINAL_APPROVED by SUPERVISOR)"
1,st_step 86793_0,STAFF MEMBER,Declaration FINAL_APPROVED by SUPERVISOR,2017-01-09 11:27:48+01:00,SUPERVISOR,declaration 86791,declaration 86791,budget 86566,declaration number 86792,26.851205,Declaration FINAL_APPROVED by SUPERVISOR,Request Payment,1326.933333,"(Declaration FINAL_APPROVED by SUPERVISOR, Request Payment)"
2,dd_declaration 86791_19,SYSTEM,Request Payment,2017-01-10 09:34:44+01:00,UNDEFINED,declaration 86791,declaration 86791,budget 86566,declaration number 86792,26.851205,Request Payment,Payment Handled,3356.633333,"(Request Payment, Payment Handled)"
3,dd_declaration 86791_20,SYSTEM,Payment Handled,2017-01-12 17:31:22+01:00,UNDEFINED,declaration 86791,declaration 86791,budget 86566,declaration number 86792,26.851205,-,-,0.000000,-
4,st_step 86798_0,STAFF MEMBER,Declaration SUBMITTED by EMPLOYEE,2017-01-09 10:26:14+01:00,EMPLOYEE,declaration 86795,declaration 86795,budget 86566,declaration number 86796,182.464172,Declaration SUBMITTED by EMPLOYEE,Declaration APPROVED by PRE_APPROVER,63363.116667,"(Declaration SUBMITTED by EMPLOYEE, Declaration APPROVED by PRE_APPROVER)"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56432,st_step 138363_0,STAFF MEMBER,Declaration SUBMITTED by EMPLOYEE,2018-12-29 17:50:14+01:00,EMPLOYEE,declaration 138359,declaration 138359,budget 86566,declaration number 138360,190.404576,Declaration SUBMITTED by EMPLOYEE,Declaration APPROVED by ADMINISTRATION,5.983333,"(Declaration SUBMITTED by EMPLOYEE, Declaration APPROVED by ADMINISTRATION)"
56433,st_step 138361_0,STAFF MEMBER,Declaration APPROVED by ADMINISTRATION,2018-12-29 17:56:13+01:00,ADMINISTRATION,declaration 138359,declaration 138359,budget 86566,declaration number 138360,190.404576,Declaration APPROVED by ADMINISTRATION,Declaration FINAL_APPROVED by SUPERVISOR,6659.650000,"(Declaration APPROVED by ADMINISTRATION, Declaration FINAL_APPROVED by SUPERVISOR)"
56434,st_step 138362_0,STAFF MEMBER,Declaration FINAL_APPROVED by SUPERVISOR,2019-01-03 08:55:52+01:00,SUPERVISOR,declaration 138359,declaration 138359,budget 86566,declaration number 138360,190.404576,Declaration FINAL_APPROVED by SUPERVISOR,Request Payment,7164.600000,"(Declaration FINAL_APPROVED by SUPERVISOR, Request Payment)"
56435,dd_declaration 138359_19,SYSTEM,Request Payment,2019-01-08 08:20:28+01:00,UNDEFINED,declaration 138359,declaration 138359,budget 86566,declaration number 138360,190.404576,Request Payment,Payment Handled,3430.666667,"(Request Payment, Payment Handled)"


In [ ]:
def func(x):
    x['num']=len(x['concept:name'])
    return x

In [ ]:
df_agrupado_trazas=log_ct.groupby(['case:id']).agg(lambda x: list(x))

In [ ]:
df_agrupado_trazas['num_act']=[ len(lista) for lista in df_agrupado_trazas['concept:name']]

In [ ]:
df_agrupado_trazas

In [6]:
log_ct["org:role"].unique()
#supongamos tres clusters 
log_sup=log_ct[log_ct["org:role"]=='SUPERVISOR']
log_emp=log_ct[log_ct["org:role"]=='EMPLOYEE']
log_own=log_ct[log_ct["org:role"]=='BUDGET OWNER']

In [7]:
log_ct[log_ct['org:resource']=='SYSTEM']['concept:name'].value_counts()

Payment Handled    10044
Request Payment    10040
Name: concept:name, dtype: int64

In [8]:
print("La varianza del cycle time de los eventos de Supervisor es"+" "+str(np.var(log_sup["Cycle time"])))

La varianza del cycle time de los eventos de Supervisor es 148956830.7510441


In [9]:
print("La varianza del cycle time de los eventos de Employer es"+" "+str(np.var(log_emp["Cycle time"])))

La varianza del cycle time de los eventos de Employer es 304054773.75200707


In [10]:
print("La varianza del cycle time de los eventos de Owner es"+" "+str(np.var(log_own["Cycle time"])))

La varianza del cycle time de los eventos de Owner es 12421852.302973488


In [11]:
#calcula la probabilidad de que ocurra cada actividad después de agrupar: .apply(lambda x: len(x)/len(log_sup))
group_act_sup=log_sup.groupby("concept:name").apply(lambda x: len(x)/len(log_sup))
group_act_own=log_own.groupby("concept:name").apply(lambda x: len(x)/len(log_own))

In [12]:
#dispersion
def schutzFormula(probs,numberTuples):
    q=1/numberTuples#np.mean(probs)

    #schutz=float(1-sum([prob-q for prob in probs])/(2*numberTuples*q))
    schutz=float(sum([abs(prob-q) for prob in probs])/(2*numberTuples*q))
    return schutz    

In [13]:
group_act_own

concept:name
Declaration APPROVED by BUDGET OWNER    0.979507
Declaration REJECTED by BUDGET OWNER    0.020493
dtype: float64

In [14]:
schutzFormula(group_act_sup.values,len(group_act_sup.values))

0.6384652278177458

In [15]:
schutzFormula(group_act_own.values,len(group_act_own.values))

0.4795067731851337

In [16]:
group_res_ct=log_ct.groupby("org:resource").apply(lambda x: np.mean(x['Cycle time']))

In [17]:
np.var(group_res_ct)

157473.63532723146

In [18]:
#Conciseness:
#results_conciseness_res=log_ct.groupby("org:resource").apply(lambda x: len(x)/len(log_ct))
df_filtrado=filterCasesByActivities(log_ct,"concept:name","case:id",['Declaration APPROVED by BUDGET OWNER','Declaration SUBMITTED by EMPLOYEE'])

In [19]:
df_filtrado2=filterCasesByActivities(log_ct,"concept:name","case:id",['Declaration REJECTED by ADMINISTRATION'])

In [20]:
df_filtrado3=filterCasesByActivities(log_ct,"concept:name","case:id",['Declaration FOR_APPROVAL by PRE_APPROVER'])

In [21]:
conciseness=[len(log_ct)/len(df_filtrado),len(log_ct)/len(df_filtrado2),len(log_ct)/len(df_filtrado3)]

In [22]:
def normBoxScore(data):

    #first the boxcox function tries to normalize the data
    boxcox=stats.boxcox(data)
    
    #after that the zscore function is also applied to reduce the scale of the data 
    results_normalized=stats.zscore(boxcox[0])
    
    return results_normalized
    

In [23]:
norm_var_ct=normBoxScore([148956830.7510441,304054773.75200707,12421852.302973488])

In [24]:
norm_schutz=normBoxScore([5.55,2.77])

In [25]:
norm_conciseness=normBoxScore(conciseness)

In [26]:
norm_var_ct

array([ 0.2354198 ,  1.08994615, -1.32536595])

In [27]:
norm_schutz

array([ 1., -1.])

In [28]:
norm_conciseness

array([-1.0875973 , -0.23904813,  1.32664543])

In [29]:
normBoxScore([0.55,0.83,1.76,75454])

array([-0.89848359, -0.61054922, -0.16237151,  1.67140432])

In [30]:
from math import log2
def kl_divergence(p, q):
    return sum(p[i] * log2(p[i]/q[i]) for i in range(len(p)))

In [31]:
ids=['declaration 86791', 'declaration 86795', 'declaration 86800']
log_ids=log_ct[log_ct["case:concept:name"].isin(ids)]
val_log_ids=log_ids['concept:name'].value_counts()
prob_log_ids=val_log_ids/sum(val_log_ids)
prob_log_ids

Declaration FINAL_APPROVED by SUPERVISOR    0.214286
Request Payment                             0.214286
Declaration SUBMITTED by EMPLOYEE           0.214286
Payment Handled                             0.214286
Declaration APPROVED by PRE_APPROVER        0.142857
Name: concept:name, dtype: float64

In [32]:
ids2=['declaration 138349', 'declaration 138354', 'declaration 138359']
log_ids2=log_ct[log_ct["case:concept:name"].isin(ids2)]
val_log_ids2=log_ids2['concept:name'].value_counts()
prob_log_ids2=val_log_ids2/sum(val_log_ids2)
prob_log_ids2

Declaration SUBMITTED by EMPLOYEE           0.2
Declaration FINAL_APPROVED by SUPERVISOR    0.2
Request Payment                             0.2
Declaration APPROVED by ADMINISTRATION      0.2
Payment Handled                             0.2
Name: concept:name, dtype: float64

In [33]:
np.var(prob_log_ids)

0.0008163265306122448

In [34]:
log_ids.columns

Index(['id', 'org:resource', 'concept:name', 'time:timestamp', 'org:role',
       'case:id', 'case:concept:name', 'case:BudgetNumber',
       'case:DeclarationNumber', 'case:Amount', 'Initial Activity',
       'Final Activity', 'Cycle time', 'Transition'],
      dtype='object')

In [1]:
#prueba directa sin adaptar trazas y eso
lista=compute_interestingness_kl_divergence(log_ct,log_ids2,['org:resource','Cycle time','case:Amount','concept:name'],False)
lista

NameError: name 'compute_interestingness_kl_divergence' is not defined

In [41]:
from operator import itemgetter
max(lista,key=itemgetter(1))

('case:Amount', 5.898383626283723)

In [ ]:
kl_divergence(prob_log_ids,[0.2,0.2,0.2,0.2,0.2])

In [ ]:
kl_divergence(prob_log_ids,[0.2,0.2,0.2,0.2,0.2,0,0,0])

In [ ]:
kl_divergence(prob_log_ids,[0.1,0.3,0.5,0.05,0.05,0,0,0])